# Stable Mariages

For this workshop we'll implement a version of the **Nobel Prize-winning algorithm** for the stable marriage problem. It's called the [Gale-Shapley](https://en.wikipedia.org/wiki/Gale%E2%80%93Shapley_algorithm) algorithm AKA the "Deferred Acceptance" algorithm.

The Gale Shapley algorithm solves what's called the **Stable Marriage** Problem. Here is the problem framed as marrying men and women: 

> Given $n$ men and $n$ women, where each person has ranked all members of the opposite sex in order of preference, marry the men and women together such that there are no two people of opposite sex who would both rather have each other than their current partners. When there are no such pairs of people, the set of marriages is deemed stable.

This little model however is generally applied to more than marrying men and women. For example, it's the algorithm used to match newly graduated doctors with hospitals, and a variation on it matches kidney patients with organ donors in our hospital system.

The algorithmic question is, given lists of preferences as input, can we find a stable marriage? Can we even guarantee a stable marriage will exist for any set of preferences? The answer to both questions is yes, and it uses an algorithm called deferred acceptance.

Here is an informal description of the algorithm. It goes in rounds. In each round, each man “proposes” to the highest-preferred woman that has not yet rejected him. On the other side, each woman holds a reference to a man at all times. If a woman gets new proposals in a round, she immediately rejects every proposer except her most preferred, but does not accept that proposal. She “defers” the acceptance of the proposal until the very end.

![](Gale-Shapley.gif)

# Your Task

Build a solution `gale_shapley(men, women) -> dict` Where the input is a list of suitors and a list of Suiteds with every one in these lists holding their lists of preferences. The output is a dictionary `suitor -> suited`

Here is a possible way to design a solution:

### Man Class

The `Man(id, preference_list)` class holds the following attributes:

- Its own ID (an `int`)

- A list or array of IDs, which are ordered. So `preference_list[0]` is prefered to `preference_list[1]` and so on

- A method `Suitor.preference()` which points to its current possible reference. It should start by pointing at `preference_list[0]` and every time the Suitor gets rejected in the algorithm, this should point to the next preference.

### Woman Class

The `Woman(id, preference_list)` class holds the following attributes:

- Its own ID (an `int`)

- A list or array of IDs, which are ordered. So `preference_list[0]` is prefered to `preference_list[1]` and so on

- A set of current suitors

- A method `Suited.reject()` which returns all current suitors except the most preferred one

### The Stable Mariage Algorithm

Takes in a list of men and women, loops over suitors trying to find a match until there aren't any unassigned suitors left. Here is the pseudocode for the algorithm:

```
algorithm stable_matching
    Initialize all m ∈ M and w ∈ W to free
    while ∃ free man m who still has a woman w to propose to do
        w := first woman on m's list to whom m has not yet proposed
        if w is free then
            (m, w) become engaged
        else some pair (m', w) already exists
            if w prefers m to m' then
                m' becomes free
                (m, w) become engaged 
            else
                (m', w) remain engaged
            end if
        end if
    repeat
```


Here is some example code of a solution working:

```
men = [
    Man(id=0, preference_list=[0,1,2,3]),
    Man(id=1, preference_list=[2,3,0,1]),
    Man(id=2, preference_list=[1,0,3,2]),
    Man(id=3, preference_list=[3,2,1,0]),
]

women = [
    Woman(id=0, preference_list=[0,1,2,3]),
    Woman(id=1, preference_list=[2,3,0,1]),
    Woman(id=2, preference_list=[1,0,3,2]),
    Woman(id=3, preference_list=[3,2,1,0]),
]

stable_marriage(men, women)
```

output:

```
{0: 0, 
 2: 1, 
 1: 2, 
 3: 3}
```

In [34]:
from collections import defaultdict

class Matcher:

    def __init__(self, men, women): 
        """
        Constructs a Matcher Instance

        Take a dict of men's spousal preferences, 
        and a dict of women's spousal preferences. 
        """
        self.M = men 
        self.W = women 
        self.wives = {}
        self.pairs = []

        #index spousal preference at initialization 
        self.mrank = defaultdict(dict) #is m's ranking of w 
        self.wrank = defaultdict(dict) # is w's ranking of m 

        for m, prefs in men.items():
            for i, w in enumerate(prefs):
                self.mrank[m][w] = i 
        
        for w, prefs in women.items():
            for i, m in enumerate(prefs):
                self.wrank[w][m] = i 

        
    def __call__(self):
        return self.match()

    def prefers(self, w, m, h):
        return self.wrank[w][m] < self.wrank[w][h]
    
    def after(self, m, w):
        #return the woman favoed by m after w
        i = self.mrank[m][w] + 1 #index of woman following w in list of prefs 
        return self.M[m][i]
    
    def match(self, men=None, next=None, wives=None):
        #try to match men with their preferrred spouse. 
        if men is None: 
            men = self.M.keys() #get the completed list of men 
        if next is None: 
            #if not defined, map each man to their first preference
            next = dict((m, rank[0]) for m, rank in self.M.items())
        if wives is None: 
            wives = {}
        
        if not len(men):
            self.pairs = [(h, w) for w, h in wives.items()]
            self.wives = wives
            return wives 
        
        m, men = list(men)[0], list(men)[1:]
        w = next[m] # next woman for m to propose to
        next[m] = self.after(m, w) #woman after w in m's list of prefs 
        if w in wives:
            h = wives[w] #current husband 
            if self.prefers(w, m, h):
                men.append(h) #husband becomes available again 
                wives[w] = m # m remains unmarried
            else:
                men.append(m) # m remains unmarried 
        else:
            wives[w] = m  #w become wife of m 
        return self.match(men, next, wives)
        
        def is_stable(self, wives=None, verbose=None):
            if wives is None:
                wives = self.wives 
            for w, m in wives.items():
                i = self.M[m].index(w)
                preferrred = self.M[m][:i]
                for p in preferrred:
                    h = wives[p]
                    if self.W[p].index(m) < self.W[p].index(h):
                        msg =   "{}'s marriage to {} is unstable: " + \
                                "{} prefers {} over {} and {} prefers " + \
                                "{} over her current husband {}"
                        if verbose:
                            print (msg.format(m, w, m, p, w, p, m, h))
                        return False
        return True
# the men and their list of ordered spousal preferences
M = dict((m, prefs.split(', ')) for [m, prefs] in (line.rstrip().split(': ')
                                for line in open('../men.txt')))

# the women and their list of ordered spousal preferences
W = dict((m, prefs.split(', ')) for [m, prefs] in (line.rstrip().split(': ')
                                for line in open('../women.txt')))

match = Matcher(M, W)


# check if the mapping of wives to husbands is stable
def is_stable(wives, verbose=False):
    for w, m in wives.items():
        i = M[m].index(w)
        preferred = M[m][:i]
        for p in preferred:
            h = wives[p]
            if W[p].index(m) < W[p].index(h):  
                msg = "{}'s marriage to {} is unstable: " + \
                      "{} prefers {} over {} and {} prefers " + \
                      "{} over her current husband {}"
                if verbose:
                    print(msg.format(m, w, m, p, w, p, m, h))
                return False
    return True

# match men and women; returns a mapping of wives to husbands
wives = match()
assert is_stable(wives)             # should be a stable matching
wives


{'abi': 'jon',
 'cath': 'bob',
 'hope': 'ian',
 'ivy': 'abe',
 'jan': 'ed',
 'bea': 'fred',
 'gay': 'gav',
 'eve': 'hal',
 'dee': 'col',
 'fay': 'dan'}